# Imports

In [ ]:
from canvasapi import Canvas # pip install canvasapi

from autocanvas.config import INPUT_DIR

from autocanvas.core.conversions import (
    series_from_api_object, 
    df_from_api_list)

from autocanvas.core.course_info import (
    get_PHY_course, 
    get_assignment_group_from_name, 
    get_teaching_personel,
    get_students_from_sections,)

from autocanvas.core.assignments import get_assignment_collection

from os.path import join
import re
import pandas as pd     # manipulate tabulated data
from datetime import datetime 

API_URL = "https://ufl.instructure.com/"
try:
    from autocanvas.config import get_API_key
    API_KEY = get_API_key()
except FileNotFoundError as e:
    print(e)
    API_KEY = input("Asking for API token")

canvas = Canvas(API_URL, API_KEY)

# Get regular quizzes

In [ ]:
course = get_PHY_course(canvas, 
                        course_code="PHY2054", 
                        semester="Spring 2021")
print(course.name)

In [ ]:
df_regular_quizzes = get_assignment_collection(course,
                             assignment_group_name="recitation quizzes",
                             name_pattern="^Quiz \d+",
                             exclude_numbers=[0,10,13],
                             add_identifier_numbers=True
                            )
df_regular_quizzes

# Import DRC Info from spreadsheet

Even if we eventually are able to get all this info directly from the registrar website , still it would be good to store the data in a similar spreadsheet, so that we download it only once.  

**Update**: we now have this info. see `ufsoc.py`. Still need to write the code that produces the spreadsheet entries

In [ ]:
excel_fname = "quiz_times_spring2021.xlsx"
excel_path = join(INPUT_DIR,excel_fname)

In [ ]:
df_drc = (pd.read_excel(excel_path, skiprows=65,
                        skipfooter=9, convert_float=True,)
                    .set_index("Student Name")
                    .filter(regex='^Quiz \d+ (start|due) time$')
                    .dropna(axis=1, how='any')
                    .applymap(lambda x: str(x.round("min")) )
        )
df_drc.head(100) # change to 100 in order to check all DRC students 

In [ ]:
type_list = ['student']

users = list(course.get_users(enrollment_type=type_list))

In [ ]:
# Get Students
df_students = df_from_api_list(users)
df_students.head(10)

In [ ]:
df_full = pd.merge(left=df_drc, right=df_students, how="left", 
                  left_index=True, right_on="name", validate="1:1")
df_full

In [ ]:
# TODO: eventually we would like for each existing override that all 
# its details are consistent with the current spreadsheet
validate_existing_overrides = False

for row_id, assignment in df_regular_quizzes.iterrows():
    # Initial checks
#     assert assignment.name=="Test quiz", "Check everything before you disable safeguard" 
    print("\n", assignment["name"], "\n")
    assert len(assignment["name"].split()) == 2, "Expecting format `Quiz/Exam ###`"
    
    quiz_number = assignment["identifier_number"]
    column_start = "Quiz {} start time".format(quiz_number)
    column_due = "Quiz {} due time".format(quiz_number)
    assignment_object = assignment["object"]
    
    for student_id, student in df_full.iterrows():
        # the API accepts both the date format in excel and datetime object
        datetime_start = datetime.fromisoformat(student[column_start])
        datetime_due = datetime.fromisoformat(student[column_due])
        now = datetime.now()
        # making sure that the assignment is in the future
        # TODO: make it robust over timezones
        if datetime_start < now:
            print("Skipping student: {}. Requested start time ({}) is in the past."
                  .format(student["name"],
                          datetime_start.isoformat()))
            continue
        try:
            assignment_object.create_override(
                assignment_override={
                "student_ids":[student_id],
                "title":"1 Student",
                "due_at":datetime_due,
                "unlock_at":datetime_start,
                "lock_at":datetime_due
                })
            print(student["name"])
        except Exception as e:
            if '"type":"taken"' in str(e):
                print("Skipping Student: {}. Override already present."
                      .format(student["name"]))
            else:
                print(e)